In [49]:
#所需要的库
#The required libraries
import re
from bs4 import BeautifulSoup
from concurrent import futures
import sys
import time
import pandas as pd
import requests
import json
from fake_useragent import UserAgent

# Set the base parameters
HEADERS = {}
NUM_THREADS = 5
PRINT = False
proxies = None
ua = UserAgent()

class LianjiaSpider(object):
    #为防止反爬虫系统设置自动切换代理功能，实际并未使用    
    #To circumvent the anti-crawler system, the automatic switching agent function was set, which was not actually used
    def gen_proxies_from_cert(self,proxy_user, proxy_password):
        PROXY_HOST = "proxy.abuyun.com"
        PROXY_PORT = "9020"
        proxy_meta = "http://%(user)s:%(pass)s@%(host)s:%(port)s" % {
            "host" : PROXY_HOST,
            "port" : PROXY_PORT,
            "user" : proxy_user,
            "pass" : proxy_password
        }

        proxies = {
            "http"  : proxy_meta,
            "https" : proxy_meta
        }

        return proxies

    def __init__(self):
        self.headers = {"User-Agent": UserAgent().random}
        self.datas = list()
    #获取最大可爬取页数
    #Gets the maximum number of pages to climb
    def getMaxPage(self, url):
        response = requests.get(url, headers = self.headers)
        if response.status_code == 200:
            source = response.text
            soup = BeautifulSoup(source, "html.parser")
            pageData = soup.find("div", class_ = "page-box house-lst-page-box")["page-data"]
            maxPage = eval(pageData)["totalPage"]
            return  maxPage
        else:
            print("Fail status: {}".format(response.status_code))
            return None
    #连接和数据记录函数 
    #Connection and data recording functions
    def parsePage(self, url):
        dataset=pd.DataFrame(columns=["成交记录ID","户型", "小区名称",
                                                "建筑面积", "成交价", "挂牌价", "单价",
                                                "成交周期", "成交日期", "朝向", "装修",
                                                 "楼层",  "建筑类型"])
        maxPage = self.getMaxPage(url)
        for pageNum in range(1, maxPage+1 ):
            url = "https://xa.lianjia.com/chengjiao/pg{}/".format(pageNum)
            print("当前正在爬取: {}".format(url))
            detail = self.parseDetail(url)
            dataset = dataset.append(detail)
        # 将所有爬取的二手房数据存储到csv文件中
        #Store all crawling second-hand house data in a CSV file
        dataset.to_csv("Lianjia.csv", encoding='utf_8_sig', index=False)


    #数据获取函数    
    #Data acquisition function
    def parseDetail(self, url):
        #代理设置，实际连接良好，并未部署
        # Proxy setup, actually well connected, not deployed
        PROXY_USERNAME = 'H99H75AT4V8D761D'
        PROXY_PASSWORD = '9EEDC185780C0189'
        abuyun_proxies = self.gen_proxies_from_cert(PROXY_USERNAME, PROXY_PASSWORD)
        proxies = abuyun_proxies
        #数据爬取
        #Data crawl
        done = False
        exception_time = 0
        HEADERS["User-Agent"] = ua.random
        while not done:
            try:
                
                print("正在获取 {}".format(url))
                r = requests.get(url, headers=HEADERS)
                bs_obj = BeautifulSoup(r.text, "html.parser")
                done = True
            except Exception as e:
                
                print(e)
                exception_time += 1
                time.sleep(1)
                if exception_time > 10:
                    return None
        df = pd.DataFrame()
        #从数据包中读取数据
        #Read data from the package
        if bs_obj is not None:
            try:
                div_list = bs_obj.find_all("div", {"class": "info"})
                for div in div_list:
                    div_title = div.find("div", {"class": "title"}).find("a")
                    url = div_title.attrs["href"]
                    trans_id = url[url.rfind('/')+1:url.rfind('.')]
                    title_strs = div_title.get_text().split(" ")
                    xiaoqu_name = title_strs[0]
                    house_type = title_strs[1]
                    built_area = title_strs[2].replace("平米", "")
                
                    house_info_strs = div.find("div", {"class": "houseInfo"}).get_text().replace(" ", "").split("|")
                    direction = house_info_strs[0]
                    decoration = house_info_strs[1].replace("&nbsp;", "")             
                    deal_date = div.find("div", {"class": "dealDate"}).get_text()
                    deal_price = div.find("div", {"class": "totalPrice"}).find("span", {"class": "number"}).get_text()
                    position_info_strs = div.find("div", {"class": "positionInfo"}).get_text().split(" ")
                    floor = position_info_strs[0]
                    build_type = position_info_strs[1].split("年建")[1] if "年建" in position_info_strs[1] else position_info_strs[1]
                    unit_price = div.find("div", {"class": "unitPrice"}).find("span", {"class": "number"}).get_text()
                
                    span_deal_cycle = div.find("span", {"class": "dealCycleTxt"}).find_all("span")
                    if len(span_deal_cycle) == 2:
                        list_price = "".join(list(filter(lambda x: str.isdigit(x) or (x == "."), span_deal_cycle[0].get_text())))
                        deal_cycle = "".join(list(filter(str.isdigit, span_deal_cycle[1].get_text())))
                    else:
                        deal_cycle = "".join(list(filter(str.isdigit, span_deal_cycle[0].get_text())))
                        list_price = ""
                    temp_df = pd.DataFrame(data=[[trans_id,house_type,xiaoqu_name,
                                              built_area, deal_price, list_price, unit_price, 
                                              deal_cycle, deal_date, direction, decoration, 
                                              floor, build_type]], 
                                       columns=["成交记录ID","户型", "小区名称",
                                                "建筑面积", "成交价", "挂牌价", "单价",
                                                "成交周期", "成交日期", "朝向", "装修",
                                                 "楼层",  "建筑类型"])
                    df = df.append(temp_df)
            except Exception as e:
                print(e)
    
        return df
    
        
if __name__ == "__main__":
    Lianjia = LianjiaSpider()
    Lianjia.parsePage("https://xa.lianjia.com/chengjiao/")

当前正在爬取: https://xa.lianjia.com/chengjiao/pg1/
正在获取 https://xa.lianjia.com/chengjiao/pg1/
当前正在爬取: https://xa.lianjia.com/chengjiao/pg2/
正在获取 https://xa.lianjia.com/chengjiao/pg2/
当前正在爬取: https://xa.lianjia.com/chengjiao/pg3/
正在获取 https://xa.lianjia.com/chengjiao/pg3/
list index out of range
当前正在爬取: https://xa.lianjia.com/chengjiao/pg4/
正在获取 https://xa.lianjia.com/chengjiao/pg4/
当前正在爬取: https://xa.lianjia.com/chengjiao/pg5/
正在获取 https://xa.lianjia.com/chengjiao/pg5/
当前正在爬取: https://xa.lianjia.com/chengjiao/pg6/
正在获取 https://xa.lianjia.com/chengjiao/pg6/
list index out of range
当前正在爬取: https://xa.lianjia.com/chengjiao/pg7/
正在获取 https://xa.lianjia.com/chengjiao/pg7/
list index out of range
当前正在爬取: https://xa.lianjia.com/chengjiao/pg8/
正在获取 https://xa.lianjia.com/chengjiao/pg8/
当前正在爬取: https://xa.lianjia.com/chengjiao/pg9/
正在获取 https://xa.lianjia.com/chengjiao/pg9/
当前正在爬取: https://xa.lianjia.com/chengjiao/pg10/
正在获取 https://xa.lianjia.com/chengjiao/pg10/
list index out of range
当前正在爬取: http

当前正在爬取: https://xa.lianjia.com/chengjiao/pg82/
正在获取 https://xa.lianjia.com/chengjiao/pg82/
当前正在爬取: https://xa.lianjia.com/chengjiao/pg83/
正在获取 https://xa.lianjia.com/chengjiao/pg83/
list index out of range
当前正在爬取: https://xa.lianjia.com/chengjiao/pg84/
正在获取 https://xa.lianjia.com/chengjiao/pg84/
当前正在爬取: https://xa.lianjia.com/chengjiao/pg85/
正在获取 https://xa.lianjia.com/chengjiao/pg85/
list index out of range
当前正在爬取: https://xa.lianjia.com/chengjiao/pg86/
正在获取 https://xa.lianjia.com/chengjiao/pg86/
当前正在爬取: https://xa.lianjia.com/chengjiao/pg87/
正在获取 https://xa.lianjia.com/chengjiao/pg87/
当前正在爬取: https://xa.lianjia.com/chengjiao/pg88/
正在获取 https://xa.lianjia.com/chengjiao/pg88/
当前正在爬取: https://xa.lianjia.com/chengjiao/pg89/
正在获取 https://xa.lianjia.com/chengjiao/pg89/
list index out of range
当前正在爬取: https://xa.lianjia.com/chengjiao/pg90/
正在获取 https://xa.lianjia.com/chengjiao/pg90/
list index out of range
当前正在爬取: https://xa.lianjia.com/chengjiao/pg91/
正在获取 https://xa.lianjia.com/chengjiao/

In [66]:
#链接百度地图api，查询交易房屋所在小区地理位置信息并记录
#Link to API of Baidu map, query the geographic location information of the community where the transaction houses are located and record
def encode_address(add):
    BAIDU_AK = "gTiZ7d7Ivp4lM0nVFIieIUjRkoxr4PS3"
    http_url = "http://api.map.baidu.com/geocoder/v2/"
    params = {
            "address": add,
            "ak": BAIDU_AK,
            "output": "json"
        }
    try:
        ret = requests.get(http_url, params=params)
        o = json.loads(ret.text)
        if o['status'] != 0:
            print("[E]: 地址编码异常: {}，".format(o['status']) + add)
            return None
        longtitude = o['result']['location']['lng']
        latitude = o['result']['location']['lat']
        return latitude,longtitude
    except Exception as e:
        print("[E]: 地址编码错误，" + add + e)
        return None     


data = pd.read_csv("Lianjia.csv")
latitude=[]
longtitude=[]
for i in data['小区名称']:
    address="陕西省西安市"+i
    try:
        a,b=encode_address(i)
        latitude.append(a)
        longtitude.append(b)
    except Exception as e:
        a,b=0,0
        latitude.append(a)
        longtitude.append(b)
        print(e)

[E]: 地址编码异常: 1，康乐路付98号院
cannot unpack non-iterable NoneType object
[E]: 地址编码异常: 1，明德门(南西区)
cannot unpack non-iterable NoneType object
[E]: 地址编码异常: 1，永松路3号院
cannot unpack non-iterable NoneType object
[E]: 地址编码异常: 1，永松路3号院
cannot unpack non-iterable NoneType object
[E]: 地址编码异常: 1，纺建路8号
cannot unpack non-iterable NoneType object
[E]: 地址编码异常: 1，纺建路8号
cannot unpack non-iterable NoneType object
[E]: 地址编码异常: 1，互助路一号
cannot unpack non-iterable NoneType object
can only concatenate str (not "ConnectionError") to str
can only concatenate str (not "ConnectionError") to str
[E]: 地址编码异常: 1，纺建路8号
cannot unpack non-iterable NoneType object
[E]: 地址编码异常: 1，北坊巷副4号
cannot unpack non-iterable NoneType object
[E]: 地址编码异常: 1，兴工西路散排
cannot unpack non-iterable NoneType object
[E]: 地址编码异常: 1，西五路109号院
cannot unpack non-iterable NoneType object
[E]: 地址编码异常: 1，龙首北路15号
cannot unpack non-iterable NoneType object
[E]: 地址编码异常: 1，长兴路
cannot unpack non-iterable NoneType object
[E]: 地址编码异常: 1，永乐路66号
cannot unpack non-ite

In [68]:
#经纬度信息记录
#Longitude and latitude information record
data["latitude"]=latitude
data["longtitude"]=longtitude
#简单的数据清洗工作
#Simple data cleaning
fun=lambda x: x.split('(')
a1=data["楼层"].apply(fun)
a=[]
b=[]
for i in a1:
    try:
        begin=i[1].find('共')
        end=i[1].rfind('层')
        high=i[1][begin+1:end]
        a.append(i[0])
        b.append(high)
    except Exception as e:
        high="未知"
        a.append(i[0])
        b.append(high)
        print(e)
data["楼层位置"]=a
data["楼高"]=b
del data["楼层"]
data.to_csv("Lianjia2.csv", encoding='utf_8_sig', index=False)